In [1]:
%load_ext autoreload
%load_ext dotenv
%dotenv
# !huggingface-cli login --token $HUGGING_FACE_TOKEN

In [ ]:
# %%capture
# %pip install dspy

In [2]:
from dspy.retrieve.you_rm import YouRM

# colbert_retriever = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

you_retriever = YouRM()
results = you_retriever.forward('What is the latest confirmed valuation of Perplexity. Include date, raised amount, the series name of the funding round and VCs')
results[:2]

[{'long_text': 'Perplexity, the AI search engine startup, is a hot property at the moment. TechCrunch has learned that the company is currently raising at least $250 million more at a valuation of between $2.5 billion and $3 billion. The news comes on the heels of two other big fundraises that have seen the company’s valuation leapfrog in the last four months: In January, the company raised nearly $74 million at a valuation of $540 million (up from $121 million in April 2023).'},
 {'long_text': 'And at the beginning of March, the company closed funding on a valuation of $1 billion, with CEO Aravind Srinivas clarifying on Twitter today that the amount raised was roughly $63 million. Those two reported rounds are not the full story. We understand from multiple sources close to the company that Perplexity is raising a further round of at least $250 million to capitalize on the attention it’s getting in the market.'}]

In [3]:
import dspy
from dspy.evaluate import Evaluate
from dspy.retrieve.you_rm import YouRM

gpt3_model = dspy.OpenAI('gpt-3.5-turbo-0125', max_tokens=1000)

dspy.configure(lm=gpt3_model, rm=you_retriever)
# dspy.configure(lm=gpt3_model)

In [9]:
%%script echo skip

from dspy.datasets.hotpotqa import HotPotQA

dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=300, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train[0:150]]
valset = [x.with_inputs('question') for x in dataset.train[150:200]]
devset = [x.with_inputs('question') for x in dataset.dev]

# show an example datapoint; it's just a question-answer pair
trainset
# trainset[0]

skip


In [98]:
from dspy.primitives import Example

trainset = [
    Example(
        {
            "companiesToCompare": "Anthropic, Cohere",
            "valuationComparison": "Anthropic has a higher valuation ($18.4B) than Cohere ($5B)",
            "numberOfEmployeesComparison": "Cohere has much more employees (275) than Anthropic (75)",
            "yearOfFoundation": "Cohere was founded earlier (2019) than Anthropic (2021)",
            "peoplesPerspective": "People prefer Anthropic over Cohere for speed and models perfromance. People prefer Cohere over Anthropic for affordability, better ranking",
        }
    ),
]
trainset = [x.with_inputs("question") for x in trainset]
print(trainset[:1])

[Example({'question': 'What is the latest confirmed valuation of Perplexity. Include date, raised amount, the series name of the funding round and VCs', 'company': 'Perplexity', 'valuation': '$1B', 'date': 'April 2024', 'raised': '$62.7M', 'round': 'Series B1', 'VCs': 'Daniel Gross'}) (input_keys={'question'})]
[Example({'question': 'What is the latest confirmed valuation of Stability AI. Include date, raised amount, the series name of the funding round and VCs', 'company': 'Stability AI', 'valuation': '$1B', 'date': 'September 2022', 'raised': '$101M', 'round': '', 'VCs': 'Coatue, Lightspeed Venture Partners'}) (input_keys={'question'})]
[Example({'question': 'What is the latest confirmed valuation of Runway. Include date, raised amount, the series name of the funding round and VCs', 'company': 'Runway'}) (input_keys={'question'})]
